# Práca s modelom Faster R-CNN implementovaným pomocou knižnice Detectron2

### Poznámka: Práca na tomto notebooku bola vykonaná v prostredí Google Colab s jazykom Python verzie 3.10. Ak chcete pracovať na svojom zariadení, možno budete potrebovať nainštalovať nástroje [C++ Build Tools](https://visualstudio.microsoft.com/downloads/#build-tools-for-visual-studio-2019)

Stiahnutie knižnice Detectron2 a všetkych ostatnych knižnic

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install optuna

In [2]:
# detectron2 imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, LVISEvaluator
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import ColorMode

# other libs (you can remove unnecessary imports)

import torch, torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image
import cv2
import numpy as np
import IPython
import json
import os
import json
import csv
import time
#import random
from pathlib import Path

# Príprava dát
Konverzia anotačných súborov vo formáte YOLO do formátu COCO.

Funkcia na vytvorenie dvojíc obrázkov a anotácií na trénovanie Detectron2.

In [3]:
def create_data_pairs(input_path, detectron_img_path, detectron_annot_path, dir_type = 'train'):

    img_paths = Path(input_path + dir_type + '/images/').glob('*.jpg')

    pairs = []
    for img_path in img_paths:

        file_name_tmp = str(img_path).split('/')[-1].split('.')
        file_name_tmp.pop(-1)
        file_name = '.'.join((file_name_tmp))

        label_path = Path(input_path + dir_type + '/labels/' + file_name + '.txt')

        if label_path.is_file():

            line_img = detectron_img_path + dir_type+'/images/'+ file_name + '.jpg'
            line_annot = detectron_annot_path+dir_type+'/labels/' + file_name + '.txt'
            pairs.append([line_img, line_annot])

    return pairs

Uvádza cesty pre súbor údajov.

In [7]:
input_path = './Meteory_format_YOLOv5_aug/'

detectron_img_path = './Meteory_format_YOLOv5_aug/'
detectron_annot_path = './Meteory_format_YOLOv5_aug/'

Generovanie dvojíc trénovacích a validačných údajov pomocou predtým definovanej funkcie.

In [8]:
train = create_data_pairs(input_path, detectron_img_path, detectron_annot_path, 'train')
val = create_data_pairs(input_path, detectron_img_path, detectron_annot_path, 'valid')

Konverzia dvojíc trénovacích a validačných údajov na zoznamy vo formáte COCO.

In [9]:
def create_coco_format(data_pairs):

    data_list = []

    for i, path in enumerate(data_pairs):

        filename = path[0]

        img_h, img_w = cv2.imread(filename).shape[:2]

        img_item = {}
        img_item['file_name'] = filename
        img_item['image_id'] = i
        img_item['height']= img_h
        img_item['width']= img_w

        print(str(i), filename)


        annotations = []
        with open(path[1]) as annot_file:
            lines = annot_file.readlines()
            for line in lines:
                if line[-1]=="\n":
                  box = line[:-1].split(' ')
                else:
                  box = line.split(' ')

                class_id = box[0]
                x_c = float(box[1])
                y_c = float(box[2])
                width = float(box[3])
                height = float(box[4])

                x1 = (x_c - (width/2)) * img_w
                y1 = (y_c - (height/2)) * img_h
                x2 = (x_c + (width/2)) * img_w
                y2 = (y_c + (height/2)) * img_h

                annotation = {
                    "bbox": list(map(float,[x1, y1, x2, y2])),
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": int(class_id),
                    "iscrowd": 0
                }
                annotations.append(annotation)
            img_item["annotations"] = annotations
        data_list.append(img_item)
    return data_list

In [ ]:
train_list = create_coco_format(train)
val_list = create_coco_format(val)

Registrácia a získanie metadat tréningových a validačných súborov údajov v DatasetCatalogu Detectron2.

In [11]:
for catalog_name, file_annots in [("train", train_list), ("val", val_list)]:
    DatasetCatalog.register(catalog_name, lambda file_annots = file_annots: file_annots)
    MetadataCatalog.get(catalog_name).set(thing_classes=['meteor', 'satelite']) #Nastavenie vlastnych tried pre metadata súboru údajov

In [12]:
metadata = MetadataCatalog.get("train")

In [ ]:
MetadataCatalog.get("val")

# Detectron2: Faster R-CNN R50 FPN konfigurácia. Trénovanie modelu a optimalizácia hyperparametrov.

In [ ]:
import os
import optuna
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator
from detectron2.evaluation import inference_on_dataset
from optuna.samplers import GridSampler
from detectron2.data import MetadataCatalog

# Funkcia cieľa pre optimalizáciu Optuna
def objective(trial):
    # Načítanie konfigurácie Faster R-CNN
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

    # Nastavenie tréningových a testovacích datasetov
    cfg.DATASETS.TRAIN = ("train",)
    cfg.DATASETS.TEST = ("val",)
    cfg.DATALOADER.NUM_WORKERS = 8

    # Nastavenie počtu vzoriek pre tréning
    cfg.SOLVER.IMS_PER_BATCH = 32

    # Použitie predtrénovaných váh modelu Faster R-CNN
    cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"

    # Používanie GPU pre tréning
    cfg.MODEL.DEVICE = 'cuda'

    # Konfigurácia hyperparametrov pre optimalizáciu
    # BASE_LR: Základná rýchlosť učenia, ovplyvňuje rýchlosť aktualizácie váh počas tréningu
    # Možné hodnoty: 0.001, 0.005
    cfg.SOLVER.BASE_LR = trial.suggest_categorical('BASE_LR', [0.001, 0.005])

    # BATCH_SIZE_PER_IMAGE: Počet príkladov na jeden obrázok pri tréningu
    # Vyššie hodnoty môžu zlepšiť generalizáciu, ale môžu tiež spôsobiť pretrénovanie
    # Možné hodnoty: 256, 512
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = trial.suggest_categorical('BATCH_SIZE_PER_IMAGE', [256, 512])

    # FPN_MIN_LEVEL: Najnižšia úroveň vo Feature Pyramid Network (FPN)
    # Ovláda minimálnu úroveň detailov, ktoré sa majú použiť v FPN
    # Možné hodnoty: 2, 3
    cfg.MODEL.FPN.MIN_LEVEL = trial.suggest_categorical('FPN_MIN_LEVEL', [2, 3])

    # BOX_REG_LOSS_TYPE: Typ straty pri regresii obmedzujúceho boxu
    # 'smooth_l1': štandartna veria
    # 'giou': generalizovaný IoU, lepšia miera prekrývania
    # Možné hodnoty: 'smooth_l1', 'giou'
    cfg.MODEL.ROI_HEADS.BOX_REG_LOSS_TYPE = trial.suggest_categorical('BOX_REG_LOSS_TYPE', ['smooth_l1', 'giou'])

    # Ďalšie nastavenia modelu
    cfg.SOLVER.MAX_ITER = 80000  # Maximálny počet iterácií (epoch)
    cfg.SOLVER.CHECKPOINT_PERIOD = 5000  # Počet iterácií medzi uložením checkpointov
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get("train").thing_classes)  # Počet tried objektov
    cfg.SOLVER.STEPS = (70000, )

    # Vytvorenie adresára pre výstupy
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    # Inicializácia trénera a začatie tréningu
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Vyhodnotenie modelu na validačnom datasete pomocou COCOEvaluator
    evaluator = COCOEvaluator("val", cfg, False, output_dir="./output/")
    val_loader = build_detection_test_loader(cfg, "val")
    inference_result = inference_on_dataset(trainer.model, val_loader, evaluator)

    # Vráti priemernú presnosť (AP) pre detekciu objektov v bouding boxoch
    return inference_result["bbox"]["AP"]

# Definovanie priestoru hľadania pre GridSampler
search_space = {
    # BASE_LR: Základná rýchlosť učenia (0.001, 0.005)
    'BASE_LR': [0.001, 0.005],
    # BATCH_SIZE_PER_IMAGE: Počet vzoriek na jeden obrázok (256, 512)
    'BATCH_SIZE_PER_IMAGE': [256, 512],
    # FPN_MIN_LEVEL: Najnižšia úroveň vo Feature Pyramid Network (2, 3)
    'FPN_MIN_LEVEL': [2, 3],
    # BOX_REG_LOSS_TYPE: Typ straty pri regresii obmedzujúceho boxu ('smooth_l1', 'giou')
    'BOX_REG_LOSS_TYPE': ['smooth_l1', 'giou']
}

# Inicializácia GridSampler so zadaným priestorom hľadania
sampler = GridSampler(search_space)

# Vytvorenie štúdie Optuna s cieľom maximalizovať AP
study = optuna.create_study(direction='maximize', sampler=sampler)

# Spustenie optimalizácie s počtom pokusov rovnakým ako počet všetkých mriežok
study.optimize(objective, n_trials=len(sampler._all_grids))

# Výpis najlepších parametrov a najlepšieho AP
print("Best parameters: ", study.best_params)
print("Best AP: ", study.best_value)


# Príklad inferencií

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.DEVICE = 'cuda'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_faster_rcnn.pth" #zadanie váhy modelu
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65   # nastaviť testovací prah pre tento model
predictor = DefaultPredictor(cfg)

In [ ]:
MetadataCatalog.get(catalog_name).set(thing_classes=['meteor', 'satelite'])

In [ ]:
im = cv2.imread("./dataset/test/images/") #cesta k obrázku
outputs = predictor(im)
v = Visualizer(im, metadata=metadata, scale=1., instance_mode =  ColorMode.IMAGE)

v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
img = v.get_image()[:,:,[2,1,0]]
img = Image.fromarray(img)
plt.figure(figsize=(10, 10))
plt.imshow(img)

# Vyhodnotenie

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.DEVICE = 'cuda'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_faster_rcnn.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "val")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Inferencia na celom testovacom súbore údajov

In [ ]:
import os
import cv2
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog

from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "model/model_faster_rcnn.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

predictor = DefaultPredictor(cfg)

metadata = MetadataCatalog.get("meteory9000")
metadata.set(thing_classes=["meteor", "satelite"])

input_folder = "./Meteory_format_YOLOv5_aug/test/images" #priečinok s obrázkami z testovacej množiny
output_folder = "./output_folder_2" #adresár pre výstupy

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

def get_classified_filename(image_name, predictions):
    classes = predictions["instances"].pred_classes.cpu().numpy()
    class_names = [metadata.thing_classes[cls] for cls in classes]
    classified_suffix = "_".join(sorted(set(class_names)))
    filename, ext = os.path.splitext(image_name)
    return f"{filename}_{classified_suffix}{ext}"

for image_name in tqdm(image_files, desc="Processing Images"):
    im = cv2.imread(os.path.join(input_folder, image_name))

    outputs = predictor(im)

    v = Visualizer(im, metadata=metadata, scale=1.0, instance_mode=ColorMode.IMAGE)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    img = v.get_image()[:, :, [2, 1, 0]]

    img = Image.fromarray(img)
    classified_image_name = get_classified_filename(image_name, outputs)
    output_path = os.path.join(output_folder, classified_image_name)
    img.save(output_path)


print(f"Processing completed! Results are saved to: {output_folder}")